In [3]:
import pandas as pd
import numpy as np
import re
! pip install nltk
import nltk
nltk.download('stopwords')
# Gensim, for topic modeling
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv("Data/YoutubeVideoEssayProject.csv")
df = df.drop(columns ="Unnamed: 9")
df

,Title,Views,Likes,Date,Length,Transcript,Description,Creator,Creator Subscribers
0,David Lynch: The Treachery of Language,"275,647","14,000",3/30/18,11:10,0:00\n[Interviewer] “David Lynch has described...,David Lynch is famous for his reluctance to ve...,What's So Great About That?,"79,900"
1,CTRL+ALT+DEL | SLA:3,"2,400,000","83,000",4/27/18,34:10,"0:00\nHi, I'm Hareton Splimby, and welcome to ...","In attempting to go fast, Hareton Splimby suff...",hbomberguy,"1,290,000"
2,The Hobbit: A Long-Expected Autopsy (Part 1/2),"5,000,000","126,000",3/28/18,36:48,"0:07\nIn mid-2015, less than a year before her...",In which we look back at The Hobbit trilogy an...,Lindsay Ellis,"5,086,953"
3,Making Games Better for Gamers with Colourblin...,"370,306","19,000",8/22/18,13:55,"0:00\nVideo games are for everyone, and they c...",Video games are for everyone. But disabled peo...,Game Maker's Toolkit,"1,510,000"
4,FAKE FRIENDS EPISODE TWO: parasocial hell,"224,898","10,000",8/11/18,1:54:34,0:03\n[Shannon] Grape-kun was a real-life peng...,it's done!!!!\n\n \n\n / struccimovies \nhttp...,StrucciMovies,"46,700"
5,Incels | ContraPoints,"5,823,697","243,000",8/17/18,34:05,0:00\n[Mendelssohn: String Quartet No. 6 in F ...,Hello boys. Let's talk about bone structure.\n...,ContraPoints,"5,823,697"
6,DOOM: The Fake Outrage,"845,440","38,000",9/1/18,24:32,0:00\nhello everyone today we're going to be\n...,Countdown to the first accusation of meta-meta...,Shaun,"662,000"
7,Disney - The Magic of Animation,"610,400","45,000",10/3/18,15:47,0:00\nforeign\n0:07\nwhat is it about Disney A...,A look at the 12 principles of animation devel...,kaptainkristian,"45,000"
8,Nostalghia Critique,"106,500","5,400",11/27/18,9:11,0:07\nThere are a few things you can't do on Y...,"A reflection on cinema, self, and other nonsen...",KyleKallgreen,"80,800"
9,In Search Of A Flat Earth,"3,423,150","129,000",9/11/20,1:16:16,Prologue\n0:00\n[Laid back folk music] A few m...,Clickbait Title: The Twist at 37 Minutes Will ...,Folding Ideas,"892,000"


In [ ]:
#cleaned text in df
def clean_text(list_): 
    regex_newline = re.compile(r'\n')
    regex_timestamp = re.compile(r'\d:\d\d')
    regex_whitespace = re.compile(r'\s{2,}')
    for index, item in enumerate(list_):
        item = str(item)
        item = re.sub(regex_newline, " ", item)
        item = re.sub(regex_timestamp, " ", item)
        item = re.sub(regex_whitespace, " ", item)
        item = re.sub(r'"', "", item)
        item = re.sub(r'\s1\s', ' ', item)
        item = re.sub(r'\s2\s', ' ', item)
        list_[index] = item
    return list_
df["Transcript"] = clean_text(df["Transcript"])
df["Description"] = clean_text(df["Description"])
df

,Title,Views,Likes,Date,Length,Transcript,Description,Creator,Creator Subscribers
0,David Lynch: The Treachery of Language,"275,647","14,000",3/30/18,11:10,[Interviewer] “David Lynch has described his ...,David Lynch is famous for his reluctance to ve...,What's So Great About That?,"79,900"
1,CTRL+ALT+DEL | SLA:3,"2,400,000","83,000",4/27/18,34:10,"Hi, I'm Hareton Splimby, and welcome to Serio...","In attempting to go fast, Hareton Splimby suff...",hbomberguy,"1,290,000"
2,The Hobbit: A Long-Expected Autopsy (Part 1/2),"5,000,000","126,000",3/28/18,36:48,"In mid-2015, less than a year before her deat...",In which we look back at The Hobbit trilogy an...,Lindsay Ellis,"5,086,953"
3,Making Games Better for Gamers with Colourblin...,"370,306","19,000",8/22/18,13:55,"Video games are for everyone, and they can ev...",Video games are for everyone. But disabled peo...,Game Maker's Toolkit,"1,510,000"
4,FAKE FRIENDS EPISODE TWO: parasocial hell,"224,898","10,000",8/11/18,1:54:34,[Shannon] Grape-kun was a real-life penguin t...,it's done!!!! / struccimovies https://ko-fi.co...,StrucciMovies,"46,700"
5,Incels | ContraPoints,"5,823,697","243,000",8/17/18,34:05,[Mendelssohn: String Quartet No. 6 in F minor...,Hello boys. Let's talk about bone structure. S...,ContraPoints,"5,823,697"
6,DOOM: The Fake Outrage,"845,440","38,000",9/1/18,24:32,hello everyone today we're going to be talkin...,Countdown to the first accusation of meta-meta...,Shaun,"662,000"
7,Disney - The Magic of Animation,"610,400","45,000",10/3/18,15:47,foreign what is it about Disney Animation tha...,A look at the 12 principles of animation devel...,kaptainkristian,"45,000"
8,Nostalghia Critique,"106,500","5,400",11/27/18,9:11,There are a few things you can't do on YouTub...,"A reflection on cinema, self, and other nonsen...",KyleKallgreen,"80,800"
9,In Search Of A Flat Earth,"3,423,150","129,000",9/11/20,1:16:16,Prologue [Laid back folk music] A few minutes ...,Clickbait Title: The Twist at 37 Minutes Will ...,Folding Ideas,"892,000"


In [ ]:
# Initialize TfidfVectorizer, using English stopwords and converting words to lowercase
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)

In [ ]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Transcript']) # Generate a matrix
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out()) # Convert matrix to dataframe
tfidf_df.set_index(df['Title'], inplace=True) 
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'Title': 'document','level_1': 'term'})
tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(1)

,document,term,tfidf
306869,Bo Burnham vs. Jeff Bezos,bezos,0.593870
374618,Break Bread,content,0.307921
163352,CATS & The Weird Mind of TS Eliot | An Analysis,eliot,0.510959
15631,CTRL+ALT+DEL | SLA:3,comic,0.354365
83354,DOOM: The Fake Outrage,doom,0.477031
7132,David Lynch: The Treachery of Language,lynch,0.663959
93322,Disney - The Magic of Animation,action,0.376898
58408,FAKE FRIENDS EPISODE TWO: parasocial hell,grape,0.437322
230324,"Fallout: New Vegas Is Genius, And Here's Why",fallout,0.400686
442363,Fixing My Brain with Automated Therapy,eliza,0.331601


In [ ]:
!pip install altair

In [ ]:
# Some fancy visualizations to highlight the words with highest TF-IDF score in each inaugural address
import altair as alt

top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

# Terms in this list will get a red dot in the visualization
term_list = ['queer', 'peace']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)
# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600)

alt.LayerChart(...)